In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

In [3]:
import numpy as np

# 70で作成した行列をロード
X_train = np.load("./matrix/x_train.npy")
Y_train = np.load("./matrix/y_train.npy")
X_test = np.load("./matrix/x_test.npy")
Y_test = np.load("./matrix/y_test.npy")

In [4]:
X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor =torch.from_numpy(Y_train)
train_datasets = TextDataset(X_train, Y_train)
train_dataloader = DataLoader(train_datasets, shuffle=True, batch_size=64)
X_test_tensor = torch.from_numpy(X_test)
Y_test_tensor =torch.from_numpy(Y_test)
test_datasets = TextDataset(X_test, Y_test)
test_dataloader = DataLoader(test_datasets, shuffle=True, batch_size=64)

In [5]:

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(300, 4),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.layer(x)
        return logits

model = torch.load("model_73.pth")  #loadするときは同様のmodelを定義してある必要がある

In [6]:
def test_loop(dataloader, model, data_type):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(torch.float)
            pred = model(X)
            correct += (pred.argmax(dim=1) == y.argmax(dim=1)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"{data_type} Accuracy: {(100*correct):>0.1f}%")

In [7]:
test_loop(train_dataloader, model, "train")
test_loop(test_dataloader, model, "test")

train Accuracy: 63.7%
test Accuracy: 64.1%


/Users/tajirimanato/.pyenv/versions/3.12.2/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
